# Assignment 3

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- sklearn [link](https://scikit-learn.org/)
- Keras [link](https://keras.io/guides/)

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q3.ipynb`.


## Question 3
Fake news is a widespread problem and there are many methods for combating it.
You have to build a fake news detection system using a ML model. Train any ML model (ANN, LSTM) over the given Dataset.
The dataset has short statements spoken by people and has the meta-information and corresponding label for those sentences. 
Your target is label column which has 6 labels(in the increasing order of truthfullness): pants-fire, false, barely-true, half-true, mostly-true, true.

The features are 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c', 'venue' and the target is column "label".

The statement is made by speaker whose job, party are given along with 6 columns which are an account of the  type of news(labels) the person has shared before. 
The person who has shared fake content before is likely to share it in future and this can be accounted by the ML model as a feature. Column barely_true_c contains how many barely_true news has the speaker shared (and so is with column X_c, value of X_c is number of X the person shared).


You have to perform two tasks:
* task1: Binary classification <br>
Classify the given news as true/false. Take the labels pants-fire, false, barely-true as false and rest (half-true, mostly-true, true) as true.
* task2: Six-way classification <br>
Classify the given news into six-classes "pants-fire, false, barely-true, half-true, mostly-true, true".

For each of the tasks:
1) Experiment with depth of network and try to fine-tune hyperparameters reporting your observations. <br>
2) Report the accuracy, f1-score, confusion matrix on train, val and test sets. <br>
3) Experiment with bag-of-words, glove and bert embeddings(code given in the below notebook) and report results. <br> Comment on what is the affect of embedding on the results.

The pre-processing code is provided, you need to write the training and test.

Note: You are supposed to train on trainset, fine-tune on val and just eval on test set. If found that you trained on val/test sets, the penalty will be incurred.

In [ ]:
# !pip install numpy
# !pip install tensorflow
# !pip install re
# !pip install nltk
# !pip install keras
# !pip install sklearn

In [ ]:
# Importing libraries
import numpy as np
import pandas as pd
from tensorflow import keras  #feel free to use any other library
import numpy as np

import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils import np_utils


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
train = pd.read_csv('train.csv')
val = pd.read_csv('val.csv')
test = pd.read_csv('test.csv')

In [ ]:
# Dropping the 'id' column
train.drop('id', axis = 1, inplace = True)
test.drop('id', axis = 1, inplace = True)
val.drop('id', axis = 1, inplace = True)

In [ ]:
train.head(5)

,label,statement,subject,speaker,job,state,party,barely_true_c,false_c,half_true_c,mostly_true_c,pants_on_fire_c,venue
0,False,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0,0,0,a mailer
1,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0,0,1,1,0,a floor speech.
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70,71,160,163,9,Denver
3,False,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7,19,3,5,44,a news release
4,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15,9,20,19,2,an interview on CNN


In [ ]:
# Checking the shape of data
print(train.shape)
print(val.shape)
print(test.shape)

(10269, 13)
(1284, 13)
(1283, 13)


## Clean and pre-process data
* Replace missing values
* Remove numbers and special characters
* Convert to upper-case

We experiment with two types of processing, one directly appending the other attributes like subject, job, state, party to sentence and then applying bag of words on it.

Other being encoding sentence with glove embeddings and passing just that.

In [ ]:

def dataPreprocessing(data):
    '''Function for cleaning the dataset
    '''
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = statement.split()
        
        # you can experiment with any other stemmers
        ps = PorterStemmer()
        statement = [ps.stem(word) for word in statement if not word in set(stopwords.words('english'))] # Stemming the dataset and removing stopwords
        statement = ' '.join(statement)
        subject = data['subject'][x].replace(',', ' ')
        speaker = data['speaker'][x]
        job = data['job'][x].lower()
        # job = job.replace(' ', '-')
        state = data['state'][x].lower()
        party = data['party'][x].lower()
        corpus.append(statement + ' '  + subject + ' ' + job + ' ' + state + ' ' + party)
    return corpus

In [ ]:
x_train = dataPreprocessing(train)
x_val = dataPreprocessing(val) 
x_test = dataPreprocessing(test) 

In [ ]:
len(x_train), len(x_val), len(x_test)

(10269, 1284, 1283)

In [ ]:
corpus = x_train + x_val + x_test

## Using bag-of-words embedding


In [ ]:
# Converting the corpus into bag-of-words
cv = CountVectorizer(max_features = 8000)
X = cv.fit_transform(corpus).toarray()

In [ ]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
X.shape

(12836, 8000)

In [ ]:
train.columns

Index(['label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
       'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c', 'venue'],
      dtype='object')

In [ ]:
# Selecting the columns 'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c'
label_cols = ['barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c']
x_train2 = train[label_cols]
x_val2 = val[label_cols]
x_test2 = test[label_cols]

In [ ]:
# Stacking x_train and x_train2 horizontally
x_train_bow = np.hstack((X[:len(x_train)], x_train2))
x_val_bow = np.hstack((X[len(x_train):len(x_train)+len(x_val)], x_val2))
x_test_bow = np.hstack((X[len(x_train)+len(x_val):], x_test2))

In [ ]:
x_train_bow.shape

(10269, 8005)

## Use of Glove Embedding


download glove embeddings from 'https://nlp.stanford.edu/data/glove.6B.zip','glove.6B.zip'
and place in your current working folder


In [ ]:
from google.colab import drive
drive.mount('/content/drive/MyDrive/glove.6B.zip')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/glove.6B.zip" -d "glove"

Archive:  /content/drive/MyDrive/glove.6B.zip
replace glove/glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
emmbed_dict = {}
with open('glove/glove.6B.200d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector


In [ ]:
emmbed_dict['oov'] = np.zeros(200)

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 3.7 MB/s 
     |████████████████████████████████| 4.0 MB 29.8 MB/s 
     |████████████████████████████████| 1.2 MB 42.5 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 895 kB 50.9 MB/s 
     |████████████████████████████████| 596 kB 43.0 MB/s 
     |████████████████████████████████| 6.6 MB 35.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=838ba3b5c9f61759e7de72519c0b4a8712e6842158efca0b6242a0cd604baea7
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def dataPreprocessing_glove(data):
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = word_tokenize(statement)

        embed_statement = []
        for w in statement:
            if w in emmbed_dict:
                embed_statement.append(emmbed_dict[w])
            else:
                embed_statement.append(emmbed_dict['oov'])
         
        list_200 = [0]*200 
        if len(embed_statement)<40:
          for i in range(40-len(embed_statement)):
            embed_statement.append(list_200)
        else:
          embed_statement = embed_statement[:40]
        
        # bonus: Think how you can encode the below features(hint: look upon label encoding or training your own word2vec or any other embedding model)
    
#         subject = data['subject'][x].replace(',', ' ')
#         speaker = data['speaker'][x]
#         job = data['job'][x].lower()
#         # job = job.replace(' ', '-')
#         state = data['state'][x].lower()
#         party = data['party'][x].lower()
        corpus.append(embed_statement)
    
    return np.array(corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
x_train_glove = dataPreprocessing_glove(train)
x_val_glove = dataPreprocessing_glove(val) 
x_test_glove = dataPreprocessing_glove(test) 
x_train_glove.shape

(10269, 40, 200)

In [ ]:
x_train_glove = np.hstack((x_train_glove.reshape(-1,8000), x_train2))
x_val_glove = np.hstack((x_val_glove.reshape(-1,8000), x_val2))
x_test_glove = np.hstack((x_test_glove.reshape(-1,8000), x_test2))

## Use of bert embeddings
note: we used our pre-processed code for bow which has the attributed appended to end the end of sentence. 

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

x_train_bert = np.hstack((model.encode(x_train), x_train2))
x_val_bert = np.hstack((model.encode(x_val), x_val2))
x_test_bert = np.hstack((model.encode(x_test), x_test2))

Now use the above 3 types of embedded inputs(bow, glove, bert embeddings) for the 2 classification tasks and compare their outputs


# Six-way classification

## Preprocessing

In [ ]:
num_classes = 6
# Preprocessing function for the labels
def categorize(data):
    y = data["label"].tolist()

    # Encoding the Dependent Variable
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)

    # Converting to binary class matrix
    y = np_utils.to_categorical(y, num_classes)
    return y

In [ ]:
y_train_six_way = categorize(train)
y_test_six_way = categorize(test)
y_val_six_way = categorize(val)

Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


## Model

In [ ]:
## write your code here
# Initialize hyperparameters
# Create model
# train
# test
# report accuracy, f1-score and confusion matrix

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix 

def get_metrics(x_test, y_test, model):
    y_test_ = np.argmax(y_test, axis=1)
    y_pred = model.predict(x_test, verbose = 0)
    y_pred = np.argmax(y_pred, axis=1)
    my_accuracy_score = accuracy_score(y_test_, y_pred)
    my_f1_score = f1_score(y_test_, y_pred, average='micro')
    my_confusion_matrix = confusion_matrix(y_test_, y_pred)
    print('Accuracy:',my_accuracy_score)
    print('F1_score:',my_f1_score)
    print('Confusion matrix:\n',my_confusion_matrix)

In [2]:
from keras import Model
from keras.layers import LSTM, Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Sequential

### Bag of Words

In [5]:
bow_model = Sequential()
bow_model.add(LSTM(units=100,input_shape=(8005, 1)))
bow_model.add(Dense(1024, activation='relu'))
bow_model.add(Dense(256, activation='relu'))
bow_model.add(Dense(128, activation='relu'))
bow_model.add(Dense(6, activation='softmax'))
bow_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 100)               40800     
                                                                 
 dense_8 (Dense)             (None, 1024)              103424    
                                                                 
 dense_9 (Dense)             (None, 256)               262400    
                                                                 
 dense_10 (Dense)            (None, 128)               32896     
                                                                 
 dense_11 (Dense)            (None, 6)                 774       
                                                                 
Total params: 440,294
Trainable params: 440,294
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
bow_model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
bow_history = bow_model.fit(x_train_bow, y_train_six_way, batch_size=64, epochs = 10, callbacks=[callback], validation_data=(x_val_bow,y_val_six_way))


Epoch 1/10
161/161 [==============================] - 158s 944ms/step - loss: 1.6318 - accuracy: 0.2777 - val_loss: 1.5043 - val_accuracy: 0.3349
Epoch 2/10
161/161 [==============================] - 155s 964ms/step - loss: 1.3927 - accuracy: 0.4060 - val_loss: 1.3768 - val_accuracy: 0.4260
Epoch 3/10
161/161 [==============================] - 152s 944ms/step - loss: 1.3396 - accuracy: 0.4258 - val_loss: 1.3429 - val_accuracy: 0.4393
Epoch 4/10
161/161 [==============================] - 156s 969ms/step - loss: 1.3282 - accuracy: 0.4285 - val_loss: 1.3349 - val_accuracy: 0.4213
Epoch 5/10
161/161 [==============================] - 152s 943ms/step - loss: 1.3207 - accuracy: 0.4338 - val_loss: 1.3200 - val_accuracy: 0.4346
Epoch 6/10
161/161 [==============================] - 152s 942ms/step - loss: 1.3148 - accuracy: 0.4311 - val_loss: 1.3101 - val_accuracy: 0.4400
Epoch 7/10
161/161 [==============================] - 155s 964ms/step - loss: 1.3030 - accuracy: 0.4334 - val_loss: 1.3013 -

In [ ]:
get_metrics(x_test_bow, y_test_six_way, bow_model)

Accuracy: 0.4286827747466875
F1_score: 0.4286827747466875
Confusion matrix:
 [[ 99   2  65  39  30  15]
 [ 24  33  43  44  64   3]
 [ 24   0 117  37  29   7]
 [ 41   1  43 123  57   2]
 [ 16   0  34  66 129   4]
 [ 17   0  16   7   3  49]]



### Glove

In [7]:
glove_model = Sequential()
glove_model.add(LSTM(units=100,input_shape=(8005, 1)))
glove_model.add(Dense(6, activation='softmax'))
glove_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 100)               40800     
                                                                 
 dense_14 (Dense)            (None, 6)                 606       
                                                                 
Total params: 41,406
Trainable params: 41,406
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#x_train_glove = np.asarray(x_train_glove).astype(np.float32)
#y_train_six_way = np.asarray(y_train_six_way).astype(np.float32)
glove_model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
glove_history = glove_model.fit(x_train_glove, y_train_six_way, batch_size=64, epochs = 5, validation_data=(x_val_glove,y_val_six_way))


Epoch 1/5
161/161 [==============================] - 157s 949ms/step - loss: 1.6467 - accuracy: 0.2907 - val_loss: 1.5985 - val_accuracy: 0.4034
Epoch 2/5
161/161 [==============================] - 151s 940ms/step - loss: 1.5260 - accuracy: 0.3996 - val_loss: 1.4501 - val_accuracy: 0.4408
Epoch 3/5
161/161 [==============================] - 151s 940ms/step - loss: 1.3755 - accuracy: 0.4466 - val_loss: 1.3268 - val_accuracy: 0.4509
Epoch 4/5
161/161 [==============================] - 151s 940ms/step - loss: 1.3698 - accuracy: 0.4400 - val_loss: 1.3920 - val_accuracy: 0.4369
Epoch 5/5
161/161 [==============================] - 155s 963ms/step - loss: 1.3646 - accuracy: 0.4349 - val_loss: 1.3419 - val_accuracy: 0.4470


In [ ]:
type(y_train_six_way)

numpy.ndarray

In [ ]:
get_metrics(x_test_glove, y_test_six_way, glove_model)

Accuracy: 0.4333593141075604
F1_score: 0.4333593141075604
Confusion matrix:
 [[133   8  19  23  45  22]
 [ 39  34  13  27  92   6]
 [ 46   0  55  25  67  21]
 [ 48   1   8 104 101   5]
 [ 28   5   8  34 168   6]
 [ 16   0   2   7   5  62]]



### Bert

In [8]:
bert_model = Sequential()
bert_model.add(LSTM(units=100,input_shape=(8005, 1)))
bert_model.add(Dense(6, activation='softmax'))
bert_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 100)               40800     
                                                                 
 dense_15 (Dense)            (None, 6)                 606       
                                                                 
Total params: 41,406
Trainable params: 41,406
Non-trainable params: 0
_________________________________________________________________


In [ ]:
bert_model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
bert_history = bert_model.fit(x_train_bert, y_train_six_way, batch_size=64, epochs = 10, validation_data=(x_val_bert,y_val_six_way))


Epoch 1/10
161/161 [==============================] - 16s 90ms/step - loss: 1.7257 - accuracy: 0.2283 - val_loss: 1.6955 - val_accuracy: 0.2617
Epoch 2/10
161/161 [==============================] - 11s 67ms/step - loss: 1.6531 - accuracy: 0.2976 - val_loss: 1.5605 - val_accuracy: 0.3458
Epoch 3/10
161/161 [==============================] - 10s 64ms/step - loss: 1.4487 - accuracy: 0.4172 - val_loss: 1.3934 - val_accuracy: 0.4245
Epoch 4/10
161/161 [==============================] - 10s 64ms/step - loss: 1.3329 - accuracy: 0.4495 - val_loss: 1.3044 - val_accuracy: 0.4579
Epoch 5/10
161/161 [==============================] - 10s 63ms/step - loss: 1.2954 - accuracy: 0.4506 - val_loss: 1.2959 - val_accuracy: 0.4455
Epoch 6/10
161/161 [==============================] - 10s 63ms/step - loss: 1.2800 - accuracy: 0.4531 - val_loss: 1.2799 - val_accuracy: 0.4470
Epoch 7/10
161/161 [==============================] - 10s 63ms/step - loss: 1.2723 - accuracy: 0.4525 - val_loss: 1.2828 - val_accuracy:

In [ ]:
get_metrics(x_test_bert, y_test_six_way, bert_model)

Accuracy: 0.4473889321901793
F1_score: 0.4473889321901793
Confusion matrix:
 [[139   2   9  43  37  20]
 [ 36  33  12  47  76   7]
 [ 40   0  59  59  41  15]
 [ 39   1   6 149  67   5]
 [ 25   0   8  73 137   6]
 [ 18   0   3   8   6  57]]


# Binary Classification

## Preprocessing

In [ ]:
num_classes = 2

In [ ]:
# Function for preprocessing labels
def dataPreprocessingBinary(data):
    y = data["label"].tolist()

    # Changing the 'half-true', 'mostly-true', barely-true', 'pants-fire' labels to True/False for Binary Classification
    for x in range(len(y)):
        if(y[x] == 'half-true'):
            y[x] = 'True'
        elif(y[x] == 'mostly-true'):
            y[x] = 'True'
        elif(y[x] == 'barely-true'):
            y[x] = 'False'
        elif(y[x] == 'pants-fire'):
            y[x] = 'False'

    # Converting the lables into binary class matrix
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)
    y = np_utils.to_categorical(y, num_classes)
    return y

In [ ]:
y_train_binary = dataPreprocessingBinary(train)
y_test_binary = dataPreprocessingBinary(test)
y_val_binary = dataPreprocessingBinary(val)

## Model
Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


In [ ]:
## write your code here
# Initialize hyperparameters
# Create model
# train
# test
# report accuracy, f1-score and confusion matrix

### Bag of Words

In [9]:
bow_model_binary = Sequential()
bow_model_binary.add(LSTM(units=100,input_shape=(8005, 1)))
bow_model_binary.add(Dense(2, activation='sigmoid'))
bow_model_binary.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 8005, 1)]         0         
                                                                 
 lstm_7 (LSTM)               (None, 100)               40800     
                                                                 
 dense_16 (Dense)            (None, 2)                 202       
                                                                 
Total params: 41,002
Trainable params: 41,002
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 100)               40800     
                                                                 
 dense_17 (Dense)            (Non

In [ ]:
bow_model_binary.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
bow_history_binary = bow_model_binary.fit(x_train_bow, y_train_binary, batch_size=64, epochs = 10, validation_data=(x_val_bow,y_val_binary))


Epoch 1/10
161/161 [==============================] - 155s 947ms/step - loss: 0.6583 - accuracy: 0.6049 - val_loss: 0.6436 - val_accuracy: 0.5935
Epoch 2/10
161/161 [==============================] - 151s 940ms/step - loss: 0.6043 - accuracy: 0.6778 - val_loss: 0.5846 - val_accuracy: 0.6947
Epoch 3/10
161/161 [==============================] - 151s 940ms/step - loss: 0.5821 - accuracy: 0.6996 - val_loss: 0.5921 - val_accuracy: 0.6955
Epoch 4/10
161/161 [==============================] - 151s 940ms/step - loss: 0.5620 - accuracy: 0.7190 - val_loss: 0.5627 - val_accuracy: 0.7002
Epoch 5/10
161/161 [==============================] - 151s 940ms/step - loss: 0.5429 - accuracy: 0.7169 - val_loss: 0.5505 - val_accuracy: 0.6893
Epoch 6/10
161/161 [==============================] - 151s 940ms/step - loss: 0.5334 - accuracy: 0.7194 - val_loss: 0.5365 - val_accuracy: 0.6978
Epoch 7/10
161/161 [==============================] - 151s 941ms/step - loss: 0.5301 - accuracy: 0.7244 - val_loss: 0.5384 -

In [ ]:
get_metrics(x_test_bow, y_test_binary, bow_model_binary)

Accuracy: 0.7295401402961809
F1_score: 0.7295401402961809
Confusion matrix:
 [[351 205]
 [142 585]]



### Glove

In [10]:
glove_model_binary = Sequential()
glove_model_binary.add(LSTM(units=100,input_shape=(8005, 1)))
glove_model_binary.add(Dense(2, activation='sigmoid'))
glove_model_binary.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 8005, 1)]         0         
                                                                 
 lstm_9 (LSTM)               (None, 100)               40800     
                                                                 
 dense_18 (Dense)            (None, 2)                 202       
                                                                 
Total params: 41,002
Trainable params: 41,002
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 100)               40800     
                                                                 
 dense_19 (Dense)            (Non

In [ ]:
glove_model_binary.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
glove_history_binary = glove_model_binary.fit(x_train_glove, y_train_binary, batch_size=64, epochs = 10, validation_data=(x_val_glove,y_val_binary))


Epoch 1/10
161/161 [==============================] - 154s 945ms/step - loss: 0.6606 - accuracy: 0.6061 - val_loss: 0.6482 - val_accuracy: 0.6090
Epoch 2/10
161/161 [==============================] - 151s 940ms/step - loss: 0.6237 - accuracy: 0.6491 - val_loss: 0.6276 - val_accuracy: 0.6877
Epoch 3/10
161/161 [==============================] - 155s 963ms/step - loss: 0.6112 - accuracy: 0.7048 - val_loss: 0.6080 - val_accuracy: 0.6869
Epoch 4/10
161/161 [==============================] - 155s 963ms/step - loss: 0.5919 - accuracy: 0.7065 - val_loss: 0.5904 - val_accuracy: 0.6916
Epoch 5/10
161/161 [==============================] - 151s 940ms/step - loss: 0.6006 - accuracy: 0.6914 - val_loss: 0.6049 - val_accuracy: 0.6947
Epoch 6/10
161/161 [==============================] - 151s 940ms/step - loss: 0.5882 - accuracy: 0.7102 - val_loss: 0.5867 - val_accuracy: 0.6893
Epoch 7/10
161/161 [==============================] - 151s 941ms/step - loss: 0.5716 - accuracy: 0.7141 - val_loss: 0.5722 -

In [ ]:
get_metrics(x_test_glove, y_test_binary, glove_model_binary)

Accuracy: 0.5853468433359315
F1_score: 0.5853468433359315
Confusion matrix:
 [[151 405]
 [127 600]]



### Bert

In [11]:
bert_model_binary = Sequential()
bert_model_binary.add(LSTM(units=100,input_shape=(389, 1)))
bert_model_binary.add(Dense(2, activation='sigmoid'))
bert_model_binary.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 100)               40800     
                                                                 
 dense_20 (Dense)            (None, 2)                 202       
                                                                 
Total params: 41,002
Trainable params: 41,002
Non-trainable params: 0
_________________________________________________________________


In [ ]:
bert_model_binary.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
bert_history_binary = bert_model_binary.fit(x_train_bert, y_train_binary, batch_size=64, epochs = 10, validation_data=(x_val_bert,y_val_binary))


Epoch 1/10
161/161 [==============================] - 16s 86ms/step - loss: 0.6558 - accuracy: 0.6113 - val_loss: 0.6404 - val_accuracy: 0.6238
Epoch 2/10
161/161 [==============================] - 11s 67ms/step - loss: 0.6012 - accuracy: 0.6738 - val_loss: 0.5858 - val_accuracy: 0.6737
Epoch 3/10
161/161 [==============================] - 10s 65ms/step - loss: 0.5595 - accuracy: 0.7076 - val_loss: 0.5523 - val_accuracy: 0.6963
Epoch 4/10
161/161 [==============================] - 10s 64ms/step - loss: 0.5416 - accuracy: 0.7136 - val_loss: 0.5342 - val_accuracy: 0.6970
Epoch 5/10
161/161 [==============================] - 10s 64ms/step - loss: 0.5334 - accuracy: 0.7137 - val_loss: 0.5438 - val_accuracy: 0.6885
Epoch 6/10
161/161 [==============================] - 10s 64ms/step - loss: 0.5269 - accuracy: 0.7189 - val_loss: 0.5436 - val_accuracy: 0.6947
Epoch 7/10
161/161 [==============================] - 10s 64ms/step - loss: 0.5233 - accuracy: 0.7198 - val_loss: 0.5319 - val_accuracy:

In [ ]:
get_metrics(x_test_bert, y_test_binary, bert_model_binary)

Accuracy: 0.7388932190179267
F1_score: 0.7388932190179267
Confusion matrix:
 [[344 212]
 [123 604]]
